In [1]:
import random
from string import ascii_uppercase, ascii_letters
from pprint import pprint
from copy import deepcopy

In [2]:
# ## from http://users.telenet.be/d.rijmenants/en/enigmatech.htm
# entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'  #(rotor right side)   
# I     = 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'
# II    = 'AJDKSIRUXBLHWTMCQGZNPYFVOE'
# III   = 'BDFHJLCPRTXVZNYEIWGAKMUSQO'
# IV    = 'ESOVPZJAYQUIRHXLNFTGKDCMWB'
# V     = 'VZBRGITYUPSDNHLXAWMJQOFECK'
# ## Additional rotors used by Kriegsmarine M3 and M4 only:
# VI    = 'JPGVOUMFYQBENHZRDKASXLICTW'
# VII   = 'NZJHGRCXMYSWBOUFAIVLPEKQDT'
# VIII  = 'FKQHTLXOCBJSPDZRAMEWNIUYGV'

# ## reflectors, standard army & airforce
# contacts   = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' 
# breflector = 'YRUHQSLDPXNGOKMIEBFZCWVJAT'
# creflector = 'FVPJIAOYEDRZXWGCTKUQSBNMHL'

{'I':'Q','II':'E','III':'V','IV':'J','V':'Z'}

Notches
Rotor    Notch 	Window 	next left rotor steps when rotor steps from/to

I        Y      Q       Q -> R
II       M      E       E -> F
III      D      V       V -> W
IV       R      J       J -> K
V        H      Z       Z -> A
VI-VIII  H+U    Z, M    Z -> A and M -> N

Rotor 	Notch 	Window 	next left rotor steps when rotor steps from/to

I
II
III
IV
V
VI VII VIII 

	

Y
M
D
R
H
H + U

	

Q
E
V
J
Z
Z and M

	

Q -> R
E -> F
V -> W
J -> K
Z -> A
Z -> A and M -> N

In [27]:
## initial data needed to generate setup of rotors through Bombe Scrambler
# entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
# raw_rotors = {'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}
# forward_rotors = {k:[ascii_uppercase.index(c) for c in raw_rotors[k]] for k in raw_rotors.keys()}
# rev_rotors = {}
# for r in raw_rotors.keys():
#     working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
#     rev_rotors[r] = [working[k] for k in sorted(working.keys())]
# reflectors = {'Brf': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 
#               'Crf': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}

entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
raw_rotors = {'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}
notches = {'I':'Q','II':'E','III':'V','IV':'J','V':'Z'}
## forward rotors is the forward in:out pairings of each rotor as the character index of the A-Z ascii alphabet stored in 'entry'
forward_rotors = {k:[ascii_uppercase.index(c) for c in raw_rotors[k]] for k in raw_rotors.keys()}

## couldn't see a straightforward list comprehension for the reverse - the in:out pairing for when the current flows
## back from the reflector to the final output. Hopefully this two-step for loop isn't too slow
rev_rotors = {}
for r in raw_rotors.keys():
    working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
    rev_rotors[r] = [working[k] for k in sorted(working.keys())]

# for k,v in rev_rotors.items():
#     print(k,':',v)

## in:out pairings for reflectors are hardcoded in, were originally generated by code
reflectors = {'Brf': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 'Crf': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}
# print(reflectors)

In [45]:
class ScramblerBombe:
    
    def __init__(self,left_rotor,middle_rotor,right_rotor,reflector,menu_link='ZZZ'):
        """rotors must be strings referring to either ['I','II','III','IV','V']
        reflector must be string, one of either ['B','C']"""
        
        self.right_rotor = right_rotor
        self.middle_rotor = middle_rotor
        self.left_rotor = left_rotor
        self.reflector = reflectors[reflector + 'rf']
        self.menu_link = menu_link
        self.middle_notch = entry.index(notches[self.middle_rotor])   ## point if right rotor reaches will trigger middle rotor to step
        self.left_notch = entry.index(notches[self.left_rotor])  ## point if middle rotor reaches will trigger left rotor to step
    
    def once_thru_scramble(self,start_character, direction='forward', first_rotor='I', pos1=0, second_rotor='II', pos2=0, 
                       third_rotor='III', pos3=0):
        """ start_character must be single ASCII character A-Z
        direction is either 'forward' or 'back' """
        if direction == 'forward':
            usedict = {k:v for k,v in forward_rotors.items()}
        elif direction == 'back':
            usedict = {k:v for k,v in rev_rotors.items()}
        else:
            print('only forward or back for direction')
            return 'wtf'

        entry = ascii_uppercase
        start_character = start_character.upper()
        entry_pos = entry.index(start_character)
        fst_pos_mod = (26 + pos1 - 0)%26
        fst_in = (entry_pos + fst_pos_mod)%26
        fst_out = usedict[first_rotor][fst_in]
        ch1o = entry[fst_out]

        scd_pos_mod = (26 + pos2 - pos1)%26
        scd_in = (fst_out + scd_pos_mod)%26
        ch2i = entry[scd_in]
        scd_out = usedict[second_rotor][scd_in]
        ch2o = entry[scd_out]

        thd_pos_mod = (26 + pos3 - pos2)%26
        thd_in = (scd_out + thd_pos_mod)%26
        ch3i = entry[thd_in]
        thd_out = usedict[third_rotor][thd_in]
        ch3o = entry[thd_out]

        if direction == 'forward':
            print(f"{start_character} -> (RR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (LR in) {ch3i} -> (LR out) {ch3o}")
        elif direction == 'back':
            print(f"{start_character} -> (LR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (RR in) {ch3i} -> (RR out) {ch3o}")

        return ch3o
    
    def full_scramble(self,in_ch, lpos=0, mpos=0, rpos=0):
        in_ch = in_ch.upper()
        left_rotor = self.left_rotor
        middle_rotor = self.middle_rotor
        right_rotor = self.right_rotor
        rflector = self.reflector
        ## first run right to left through scrambler
        forward_run = self.once_thru_scramble(in_ch, 'forward', first_rotor=right_rotor, pos1=rpos, second_rotor=middle_rotor,
                                         pos2=mpos, third_rotor=left_rotor, pos3=lpos)

        ## reflector back around for return
        rfi_pos_mod = (26 + 0 - lpos)%26    ## the '0' is there to matching formatting of other position modifiers - reflector is not moved so it will always be 0
        rf_in = (entry.index(forward_run) + rfi_pos_mod)%26
        chri = entry[rf_in]
        mirrored = rflector[chri]

        print(f"{forward_run} -> {chri} (into reflector) -> {mirrored} (reflected out)")

        ## second run back left to right thru scrambler
        back_run = self.once_thru_scramble(mirrored, direction='back', first_rotor=left_rotor, pos1=lpos, 
                                      second_rotor=middle_rotor, pos2=mpos, third_rotor=right_rotor, pos3=rpos)

        bk_out = entry.index(back_run)
        bko_pos_mod = (26 + 0 - rpos)%26  ## as above, '0' just reflects that the entry interface doesn't move
        bk_final = (bk_out + bko_pos_mod)%26
        final = entry[bk_final]
        print('RR back out:  ', back_run, '-->', final)

        return final
    
    def step_rotor(self,p_in):
        """"""
        if p_in == 25:
            p_in = 0
        else:
            p_in += 1
        return p_in
    
    def step_test(self, in_ch, lftpos=0, midpos=0, rgtpos=0):
#         lftpos, midpos, rgtpos = (ascii_uppercase.index(s) for s in startset.upper())

        mnotch = self.middle_notch   ## point if right rotor reaches will trigger middle rotor to step
        lnotch = self.left_notch  ## point if middle rotor reaches will trigger left rotor to step

        print('in:    ', in_ch)      
        rgtpos = self.step_rotor(rgtpos)
        if rgtpos == mnotch and midpos == lnotch:
            midpos = self.step_rotor(midpos)
            lftpos = self.step_rotor(lftpos)
            print('--- left & middle rotor step')
        elif rgtpos == mnotch:
            midpos = self.step_rotor(midpos)
            print('--- middle rotor step ---')

        print('rp = ', rgtpos, 'mp = ', midpos, 'lp = ', lftpos)
        out = self.full_scramble(in_ch, lpos=lftpos, mpos=midpos, rpos=rgtpos)
        print('out:   ', out,'\n')

        return out

In [46]:
eb1 = ScramblerBombe(left_rotor='II',middle_rotor='V',right_rotor='III',reflector='C')

In [47]:
eb1.middle_rotor

'V'

In [48]:
eb1.once_thru_scramble('q')

Q -> (RR out) X -> (MR in) X -> (MR out) V -> (LR in) V -> (LR out) M


'M'

In [49]:
eb1.full_scramble('q')

Q -> (RR out) I -> (MR in) I -> (MR out) U -> (LR in) U -> (LR out) P
P -> P (into reflector) -> C (reflected out)
C -> (LR out) P -> (MR in) P -> (MR out) J -> (RR in) J -> (RR out) E
RR back out:   E --> E


'E'

In [50]:
eb1.step_test('Q')

in:     Q
rp =  1 mp =  0 lp =  0
Q -> (RR out) W -> (MR in) V -> (MR out) O -> (LR in) O -> (LR out) M
M -> M (into reflector) -> X (reflected out)
X -> (LR out) I -> (MR in) I -> (MR out) F -> (RR in) G -> (RR out) S
RR back out:   S --> R
out:    R 



'R'

In [ ]:
word = 'oh what a beautiful morning oh what a beautiful day i have a wonderful feeling everthings going my way'

In [ ]:
def only_ascii(instring):
    """strips out anything that's not an ascii character (i.e a-z alphabet character), capitalises"""
    newstring = ''
    for character in instring:
        if character in ascii_letters:
            newstring += character.upper()
    return newstring

In [ ]:
only_ascii(word)